In [3]:
import backend
import json
import generate_page as page
initial_prompt = "Create a schema for a very simple twitter application"
schema_path = f'generated/schema.json'
schema_page_path= f'test/convex/schema.ts'
actions_path = 'generated/actions.json'
crud_path = 'generated/crud.ts'
backend_json = 'generated/backend.json'
backend_path = 'test/convex/backend.ts'
page_path = 'generated/page_schema.json'


In [4]:
backend.write_to_file(backend.create_schema_structure(initial_prompt), schema_path)

Here is a schema for a simple Twitter-like application:

<jsonSchema>
{
	"description": "A simple Twitter-like application that allows users to post tweets, follow other users, and view a timeline of tweets from followed users.",
	"features_description": "- User registration and login\n- Posting tweets\n- Following other users\n- Timeline view showing tweets from followed users\n- User profile pages showing their tweets",
	"schema_tables": [
		{
			"name": "users",
			"docs": "Stores user account information",
			"fields": [
				{"name": "username", "docs": "The username of the user", "type": "v.string()"},
				{"name": "email", "docs": "The email address of the user", "type": "v.string()"},
				{"name": "name", "docs": "The display name of the user", "type": "v.string()"},
				{"name": "bio", "docs": "A short user bio", "type": "v.string()"}
			]
		},
		{
			"name": "tweets",
			"docs": "Stores individual tweets posted by users",
			"fields": [
				{"name": "userId", "docs": "ID of th

In [5]:
schema = backend.read(schema_path)
backend.write_to_file(backend.create_actions(f"Create it for the following structure: \n<applicationStructure>\n{schema}\n</applicationStructure>"), actions_path)

Here is the generated JSON schema for the Twitter-like application:

<jsonSchema>
{
  "actions_description": "The application will need actions to fetch tweets for a user's timeline, post new tweets, fetch user profiles, and manage user follows.",
  "query_actions_required": "Query actions will be needed to fetch tweets and user profiles.",
  "query_actions": [
    {
      "name": "getTweets",
      "where_used": "Used to fetch tweets for a user's timeline",
      "docs": "Retrieves tweets from users that the authenticated user follows",
      "requires_auth": true,
      "arguments": [
        {
          "name": "limit",
          "docs": "Maximum number of tweets to return",
          "type": "v.optional(v.number())"
        }
      ],
      "returns": "An array of tweet objects",
      "return_type": "v.array(v.object({userId: v.id(\"users\"), text: v.string()}))",
      "workflow_steps": [
        {
          "step": "Fetch the IDs of users that the current user follows from the f

In [6]:
# entirely compiler
schema = json.loads(backend.read(schema_path))
actions = backend.read(actions_path)
schema_page = backend.create_schema(schema, schema_page_path)
crud_page = backend.create_crud(schema, crud_path)
# faker_data = backend.create_faker_data_code(schema, 'generated/faker.ts')
actions_code = backend.create_actions_code(actions, schema_page, crud_page, backend_json)
actions_page = backend.create_actions_page(json.loads(actions), json.loads(actions_code),crud_page, backend_path)

Here is the code for the requested Convex actions in JSON schema format:

<jsonSchema>
{
  "getTweets": "const followedUserIds = await getManyFollows(d, (follow) => follow.followerId.equals(user._id)).collect().map(follow => follow.followedId); const tweets = await getManyTweets(d, (tweet) => followedUserIds.includes(tweet.userId)).order(\"desc\").take(args.limit ?? 50); return tweets;",
  "getUserProfile": "const profile = await getOneUsers(d, args.userId); return profile;",
  "getUserTweets": "const tweets = await getManyTweets(d, (tweet) => tweet.userId.equals(args.userId)).order(\"desc\").take(args.limit ?? 50); return tweets;",
  "postTweet": "const tweetId = await createOneTweets(d, {userId: user._id, text: args.text}); return tweetId;", 
  "followUser": "const existingFollow = await getManyFollows(d, (follow) => follow.followerId.equals(user._id) && follow.followedId.equals(args.userId)).first(); if (existingFollow) {return existingFollow._id;} const followId = await createOneFo

In [7]:
# creating the page structure
schema = backend.read(schema_path)
actions = backend.read(actions_path)
backend.write_to_file(backend.create_page_structure("Task: " + initial_prompt + "\n\nHere is the database schema for the avaiable database objects you are to use: \n<Schemas>\n" + str(schema) + "\n</Schemas>\n\n Here are the backend queries and mutations that you can call: \n<backend>" + str(actions) + "\n</backend>\n\nNow, generate the page layout strictly in the json format described above." ), page_path)

Here is the page layout schema for the simple Twitter application:

<jsonSchema>
{
  "description": "A simple Twitter-like application that allows users to post tweets, follow other users, and view a timeline of tweets from followed users.",
  "features_description": "The main features are on the home page which shows the user's timeline of tweets from followed users. The user profile page shows the user's own tweets. The nav bar allows navigating between pages and has a button to post a new tweet.",
  "pages": [
    {
      "route_name": "home",
      "docs": "The main page showing the logged in user's timeline of tweets",
      "page_title": "Your Timeline",
      "description": "See the latest tweets from people you follow",
      "data_calls": [
        {
          "query_name": "getTweets",
          "purpose": "Fetch the tweets to display in the timeline",
          "arguments": [
            {
              "argument_name": "limit",
              "type": "number",
              

In [4]:
# alternative way to make page structure: simplified version
page.write(page_path, page.generate_component_list(schema_path, actions_path))

[
  {
    "url": "/",
    "title": "Home",
    "description": "The main timeline page showing tweets from users the logged-in user is following"
  },
  {
    "url": "/login",
    "title": "Login",
    "description": "The login page for users to sign in to their account"
  },
  {
    "url": "/register",
    "title": "Register",
    "description": "The registration page for new users to create an account"
  },
  {
    "url": "/profile/[username]",
    "title": "UserProfile",
    "description": "A user's profile page showing their bio, tweets, followers, and users they are following"
  },
  {
    "url": "/compose",
    "title": "ComposeTweet",
    "description": "A page for the logged-in user to compose and post a new tweet"
  }
]
{
  "pages": [
    {
      "url": "/",
      "title": "Home",
      "description": "The main timeline page showing tweets from users the logged-in user is following",
      "components": [
        {
          "name": "Navbar",
          "description": "Navigatio

In [2]:
# generate single page
pages = json.loads(backend.read(page_path))["pages"]
response = page.generate_page(pages, pages[0], schema_path, actions_path)

Writing the Navbar component... (enter)
<replace start=6 end=8>
export default ({}: Props) => {
  return (
    <Flex gap="4">
      <Box>
        <Link href="/" size="3" weight="bold">
          Home
        </Link>
      </Box>
      <Box>
        <Link href="/profile/[username]" size="3" weight="bold">
          Profile
        </Link>
      </Box>
      <Box>
        <Link href="/compose" size="3" weight="bold">
          Compose
        </Link>
      </Box>
    </Flex>
  );
};
</replace>
Writing the page.tsx... (enter)
Here is the edited file with the Navbar component included:

<replace start=5 end=10>
export default () => {
  const userId = useStoreUserEffect();

  return (
    <>
      <Navbar />
      <TweetFeed userId={userId} />
    </>
  );
};
</replace>

This will render the Navbar component and a TweetFeed component that will need to be created to display the tweets from users the logged-in user is following. The userId is passed to the TweetFeed component so it can fetch 

In [4]:
import os
import json

docs = []

for dir in os.listdir('radixui_docs'):
  if not dir.endswith('_meta'):
    continue
  for component_path in os.listdir(os.path.join('radixui_docs', dir)):
    with open(os.path.join('radixui_docs', dir, component_path), 'r') as f:
      component = json.loads(f.read())
    component['source'] = os.path.join(dir[:-5], component_path[:-5])
    docs.append(component)

with open('prompts/radixui_docs.txt', 'w') as f:
  f.write(json.dumps(docs, indent=2))

In [3]:
print(response['content'][0]['text'])

<replace start=0 end=9>
'use client'
import Link from 'next/link'
import { Box, Flex, Text } from '@radix-ui/react-all'

export default () => {
  return (
    <Box as="nav" p="2">
      <Flex justify="between" align="center">
        <Link href="/">
          <Text size="2" weight="bold">
            Twitter Clone
          </Text>
        </Link>
        <Flex gap="4">
          <Link href="/compose">
            <Text>Compose Tweet</Text>
          </Link>
          <Link href="/profile/[username]">
            <Text>Profile</Text>
          </Link>
          <Link href="/login">
            <Text>Login</Text>
          </Link>
        </Flex>
      </Flex>
    </Box>
  )
}
</replace>

This code creates a navigation bar with a logo linking to the home page, and links to the compose tweet page, user profile page, and login page. It uses the Box, Flex, and Text components from Radix UI to create a responsive layout. The Box component is used as the main container for the navbar, with p